## Preprocessing

In [34]:
# Importing the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential

# Read the data from charity_data.csv
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
df = pd.read_csv(url)

# Display the first few rows of the dataframe
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [35]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(columns=["EIN", "NAME"], inplace=True)

In [36]:
# Determine the number of unique values in each column.
unique_counts = df.nunique()
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [37]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = df["APPLICATION_TYPE"].value_counts()
print(app_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [38]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 500
application_types_to_replace = list(app_counts[app_counts < app_cutoff].index)

# Replace in the dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [39]:
# Look at CLASSIFICATION value counts for binning
class_counts = df["CLASSIFICATION"].value_counts()
print(class_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [40]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = df["CLASSIFICATION"].value_counts()[df["CLASSIFICATION"].value_counts() > 1]
print(class_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [41]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_cutoff = 1000
classifications_to_replace = list(class_counts_gt1[class_counts_gt1 < class_cutoff].index)
# Replace in the dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: CLASSIFICATION, dtype: int64


In [42]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df)

In [43]:
# Split our preprocessed data into our features and target arrays
X = df.drop(columns=["IS_SUCCESSFUL"])
y = df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [44]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [53]:
## New Optimization Attempt 3

# Define the model with dropout layers
nn_optimization_3 = tf.keras.models.Sequential()

# First hidden layer with dropout
nn_optimization_3.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=len(X_train_scaled[0])))
nn_optimization_3.add(tf.keras.layers.Dropout(0.5))

# Additional hidden layers with dropout
nn_optimization_3.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn_optimization_3.add(tf.keras.layers.Dropout(0.5))
nn_optimization_3.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_optimization_3.add(tf.keras.layers.Dropout(0.5))

# Output layer with sigmoid activation
nn_optimization_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [54]:
# Compile the model
nn_optimization_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [55]:
# Train the model with a higher number of epochs
fit_model_optimization_3 = nn_optimization_3.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
804/804 [==============================] - 7s 4ms/step - loss: 0.6452 - accuracy: 0.6646
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5966 - accuracy: 0.7163
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5850 - accuracy: 0.7229
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5824 - accuracy: 0.7225
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5753 - accuracy: 0.7255
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5747 - accuracy: 0.7263
Epoch 7/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5724 - accuracy: 0.7266
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5698 - accuracy: 0.7256
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5700 - accuracy: 0.7287
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5696 - accura

In [56]:
# Evaluate the model using the test data
model_loss_optimization_3, model_accuracy_optimization_3 = nn_optimization_3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss_optimization_3}, Accuracy: {model_accuracy_optimization_3}")

268/268 - 1s - loss: 0.5505 - accuracy: 0.7272 - 602ms/epoch - 2ms/step
Loss: 0.5504772663116455, Accuracy: 0.7272303104400635


In [57]:
# Export the model to an HDF5 file
nn_optimization_3.save("AlphabetSoupCharity_Optimization_3.h5")

It seems like the model's performance has improved compared to the previous attempts, reaching a higher accuracy on both the training and validation sets. However, there still seems to be a slight gap between the training and validation accuracy, indicating that there might still be room for further improvement, possibly through fine-tuning or adjusting the model architecture.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.move("AlphabetSoupCharity_Optimization_3.h5", "/content/drive/My Drive")

'/content/drive/My Drive/AlphabetSoupCharity_Optimization_2.h5'